# Refinitiv Data Platform Library for Python
## Delivery - OMMItemStream - Market Price data via callback

This notebook demonstrates how to use the OMM Item Stream interface to request streaming Quote and Trade data

## Import the library and load credentials

Credentials used by this and the other tutorials notebooks are stored in the **Configuration/credentials.ipynb** file.     

You should have edited the **Configuration/credentials.ipynb** to set your credentials as part of the **Quick Start** step.

In [1]:
from refinitiv.data.delivery import omm_stream
import datetime
import json
%run ../../Configuration/credentials.ipynb

## Open the session of your choice

Use our helper function in the Credentials notebook, **open_session(session_type)**, to create and open a session to connect to the 
- Refinitiv Data Platform directly (session_type="rdp") or via 
- Eikon 4 or Refinitiv Workspace (session_type="desktop") or via a 
- local realtime infrastructure (session_type="deployed").

You can also set a default in the **credentials** notebook

In [2]:
open_session()

<refinitiv.data.session._platform_session.Definition object at 0x21bdea963d0 {session_name='default-session'}>

## Define a function to display events

In [3]:
# Callback function to display data or status events
def display_event(eventType, event):
    currentTime = datetime.datetime.now().time()
    print("----------------------------------------------------------")
    print(">>> {} event received at {}".format(eventType, currentTime))
    print(json.dumps(event, indent=2))
    return

## Create an Item Stream and register the event callbacks

In [4]:
stream = omm_stream.Definition(
    name="EUR=", 
    fields=['BID', 'ASK','OPEN_PRC','HST_CLOSE','TIMACT'], 
    domain='MarketPrice').get_stream()

# Refresh - the first full imaage we get back from the server
stream.on_refresh(lambda item_stream, event : display_event("Refresh", event))
# Update - as and when field values change, we receive updates from the server
stream.on_update(lambda item_stream, event : display_event("Update", event))
# Status - if data goes stale or item closes, we get a status message
stream.on_status(lambda item_stream, event : display_event("Status", event))
# Other errors
stream.on_error(lambda item_stream, event : display_event("Error", event))

## Open the ItemStream
The **open()** call to open the Item Stream is a synchronous one. This means the first event (either via on_refresh(), on_status() or on_error()) can occur before the open() method returns.   
However, if we use the **open_async()** asynchronous method instead, the first event callback will be after  open_async() returns.

In [5]:
# Send request to server and open stream
stream.open()
# We should receive the initial Refresh for the current field values
# followed by updates for the fields as and when they occur

[2021-09-03 10:59:25,022] - [INFO] - [default-session] - [29484] | WebSocket 0 - OMM Protocol - PRICING
Login to websocket wss://apac-3-t3.streaming-pricing-api.refinitiv.com:443/WebSocket successful
----------------------------------------------------------
>>> Refresh event received at 10:59:25.219919
{
  "ID": 1,
  "Type": "Refresh",
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "EUR="
  },
  "State": {
    "Stream": "Open",
    "Data": "Ok"
  },
  "Qos": {
    "Timeliness": "Realtime",
    "Rate": "JitConflated"
  },
  "PermData": "AwEBUmw=",
  "SeqNumber": 29038,
  "Fields": {
    "TIMACT": "09:59:00",
    "OPEN_PRC": 1.1873,
    "HST_CLOSE": 1.1873,
    "BID": 1.1869,
    "ASK": 1.1872
  }
}


<StreamState.Open: 3>

----------------------------------------------------------
>>> Update event received at 10:59:25.816836
{
  "ID": 1,
  "Type": "Update",
  "UpdateType": "Unspecified",
  "DoNotConflate": true,
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "EUR="
  },
  "SeqNumber": 29054,
  "Fields": {
    "BID": 1.1867,
    "ASK": 1.187,
    "TIMACT": "09:59:00"
  }
}
----------------------------------------------------------
>>> Update event received at 10:59:26.847234
{
  "ID": 1,
  "Type": "Update",
  "UpdateType": "Unspecified",
  "DoNotConflate": true,
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "EUR="
  },
  "SeqNumber": 29118,
  "Fields": {
    "BID": 1.1869,
    "ASK": 1.1872,
    "TIMACT": "09:59:00"
  }
}
----------------------------------------------------------
>>> Update event received at 10:59:27.017229
{
  "ID": 1,
  "Type": "Update",
  "UpdateType": "Unspecified",
  "DoNotConflate": true,
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "EUR="
  },
  "SeqNumber": 29

## Close Stream

In [6]:
stream.close()

----------------------------------------------------------
>>> Update event received at 10:59:28.817916
{
  "ID": 1,
  "Type": "Update",
  "UpdateType": "Unspecified",
  "DoNotConflate": true,
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "EUR="
  },
  "SeqNumber": 29438,
  "Fields": {
    "BID": 1.1869,
    "ASK": 1.1872,
    "TIMACT": "09:59:00"
  }
}
----------------------------------------------------------
>>> Update event received at 10:59:28.936833
{
  "ID": 1,
  "Type": "Update",
  "UpdateType": "Unspecified",
  "DoNotConflate": true,
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "EUR="
  },
  "SeqNumber": 29502,
  "Fields": {
    "BID": 1.1867,
    "ASK": 1.1871,
    "TIMACT": "09:59:00"
  }
}


<StreamState.Closed: 1>

## Close the session

In [7]:
close_session()